<a href="https://colab.research.google.com/github/ggh-os/python-deep-learning/blob/master/Cuaderno_para_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar Selenium y descargar el controlador Chrome para la versión correspondiente
!pip install selenium
!apt-get update # Actualizar la lista de paquetes disponibles
!apt install -y chromium-chromedriver # Instalar el driver para Chrome

# Configurar el entorno para que use el navegador en modo headless
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from os import mkdir, remove
import shutil
from bs4 import BeautifulSoup
import copy
import os
import sys

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 22.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.0 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,673 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

driver.get('https://www.bolsasymercados.es/bme-exchange/es/Mercados-y-Cotizaciones/Acciones/Mercado-Continuo/Precios/mercado-continuo')
time.sleep(1)

# Aceptamos las cookies si sale la ventana
try:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(
        (By.XPATH, "/html/body/div[6]/div[2]/div/div[1]/div/div[2]/div/button[2]"))).click()
except:
    pass

# Listar todos los valores
try:
    WebDriverWait(driver, 25).until(EC.element_to_be_clickable(
        (By.XPATH, "/html/body/main/div/div/div/div[2]/nav/div/ul/li[2]/a"))).click()
except TimeoutException:
    print("No se pudo encontrar el enlace 'Todos'. Verifica el XPATH o la estructura de la Página.")
    driver.quit()
    exit()

lista_direcciones = []
lista_empresas = []
lista_volumenes = []
data = []

soup = BeautifulSoup(driver.page_source, 'html.parser')
tbody = soup.find('tbody')

# Extraer las direcciones válidas
for tr in tbody.find_all('tr'):
    td = tr.find_all('td')
    href = td[0].a.get('href')
    lista_direcciones.append("https://www.bolsasymercados.es" + href)
    row = [i.text.strip() for i in td]
    data.append(row)

for i in data:
    lista_empresas.append(i[0])
    # Manejar el caso donde el volumen es '-'
    vol_str = i[5].replace('.', '').replace('- ', '0').replace('-', '0').strip()
    if vol_str == '':
        vol_str = '0'
    lista_volumenes.append(int(vol_str))

df0 = pd.DataFrame({'Empresa': lista_empresas, 'Volumen': lista_volumenes, 'Dirección': lista_direcciones})

# Filtrar empresas con volumen no cero
df0 = df0[df0['Volumen'] != 0].reset_index(drop=True)
df0

,Empresa,Volumen,Dirección
0,ACCIONA,91566,https://www.bolsasymercados.es/bme-exchange/es...
1,ACCIONA ENER,320568,https://www.bolsasymercados.es/bme-exchange/es...
2,ACERINOX,738546,https://www.bolsasymercados.es/bme-exchange/es...
3,ACS,275592,https://www.bolsasymercados.es/bme-exchange/es...
4,ADOLFO DGUEZ,6,https://www.bolsasymercados.es/bme-exchange/es...
...,...,...,...
109,UNICAJA,5649299,https://www.bolsasymercados.es/bme-exchange/es...
110,URBAS,29297245,https://www.bolsasymercados.es/bme-exchange/es...
111,VIDRALA,21220,https://www.bolsasymercados.es/bme-exchange/es...
112,VISCOFAN,52874,https://www.bolsasymercados.es/bme-exchange/es...


In [ ]:
print(f'--- Tenemos un total de ({len(df0)}) empresas cotizadas en bolsa ---')

--- Tenemos un total de (114) empresas cotizadas en bolsa ---


In [ ]:
# Define la carpeta de destino en Google Drive
destino = '/content/drive/MyDrive/Colab Notebooks/Temporal/'

# Asegurar que la carpeta destino existe
os.makedirs(destino, exist_ok=True)

# Define las columnas para el DataFrame vacío
columnas = ['Ticker', 'Fecha', 'Hora', 'Precio', 'Volumen', 'CódigoID', 'Flags']

# Creación de la primera fila de datos                                                    # <------------- PARA QUE NO DE LO PUTOS ERRORES AL CONCATENAR DATAFRAMES VACIOS
primera_fila = ['xxxx', 'xxxx', 'xxxx', 0.0, 0, 0, 'xxxx']

for index, row in df0.iterrows():
    # Define el nombre del archivo basándose en el valor de la columna 'Empresa'
    file_name = f"{row['Empresa']}.csv"
    file_path = os.path.join(destino, file_name)

    # Comprueba si el archivo ya existe en la carpeta de destino
    if not os.path.isfile(file_path):
        # Crea un DataFrame vacío con las columnas especificadas
        empty_df = pd.DataFrame([primera_fila], columns=columnas)                         # < ------------ por la misma razón

        # Guarda el DataFrame vacío como CSV en la ruta especificada
        empty_df.to_csv(file_path, index=False)

# Ruta del archivo DATAFRAME.csv
dataframe_path = os.path.join(destino, 'DATAFRAME.csv')

# Verifica si el archivo DATAFRAME.csv ya existe
if os.path.isfile(dataframe_path):
    # Si existe, cargarlo en DATAFRAME
    DATAFRAME = pd.read_csv(dataframe_path)
    print(f"Archivo existente encontrado: {dataframe_path}. Se utilizará este archivo.")
else:
    # Si no existe, crear un DataFrame inicial y guardar como DATAFRAME.csv
    DATAFRAME = pd.DataFrame([primera_fila], columns=columnas)
    DATAFRAME.to_csv(dataframe_path, index=False)

driver = web_driver()  # Asegúrate de tener el driver correcto y el path configurado

def buscar_volumen_por_direccion(df, direccion_buscada):
    resultado = df[df['Dirección'] == direccion_buscada]['Volumen']
    if not resultado.empty:
        return resultado.iloc[0]
    else:
        return "Dirección no encontrada"

def buscar_empresa_por_direccion(df, direccion_buscada):
    resultado = df[df['Dirección'] == direccion_buscada]['Empresa']
    if not resultado.empty:
        return resultado.iloc[0]
    else:
        return "Dirección no encontrada"


def barra_de_progreso(progreso, total, contador_de_paginas):
    # Calcula el porcentaje
    porcentaje = 100 * (progreso / total) if total > 0 else 100
    largo_barra = 100  # Define el largo de la barra de progreso

    # Calcula las partes llenas y vacías de la barra
    lleno = int((porcentaje / 100) * largo_barra)
    lleno = min(lleno, largo_barra)  # Evita exceder el largo de la barra
    vacio = largo_barra - lleno
    barra = '#' * lleno + '-' * vacio

    # Determina el color según el porcentaje
    if porcentaje > 100:
        color = '\033[43m'  # Fondo amarillo
    else:
        color = '\033[0m'  # Fondo blanco

    # Imprime la barra de progreso con el color aplicado
    print(f'\r{color}[{barra}] {porcentaje:.2f}% -- Página ({contador_de_paginas})\033[0m', end='')



fallos = 0
vueltas = 1
numero = len(df0)

def process_company(direccion, vol, nombre, driver):
    driver.get(direccion)
    driver.implicitly_wait(5)
    global fallos
    global vueltas

    # Aceptamos las cookies si salen
    try:
        WebDriverWait(driver, 3).until(EC.element_to_be_clickable(
            (By.XPATH, "/html/body/div[6]/div[2]/div/div[1]/div/div[2]/div/button[2]"))).click()
    except:
        pass

    # Obtener el ticker
    try:
        Ticker = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, "/html/body/main/div/div/div/div[1]/div[2]/div/div/div[2]/div[1]/span[3]"))).text
    except TimeoutException:
        print(f"Error al obtener el ticker para {nombre}")
        return pd.DataFrame()

    # Navegar a 'Negociaciones'
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, "/html/body/main/div/div/div/div[1]/div[2]/div/div/nav/ul/li[6]/a"))).click()
    except TimeoutException:
        print(f"Error al navegar a 'Negociaciones' para {nombre}")
        return pd.DataFrame()

    ruta_archivo = f'/content/drive/MyDrive/Colab Notebooks/Temporal/{nombre}.csv'
    dfbase = pd.read_csv(ruta_archivo)

    vol_sum = dfbase['Volumen'].sum() if not dfbase.empty else 0
    contador_de_paginas = 1

    while True:
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
        except TimeoutException:
            print(f"Timeout al esperar el contenido de la Página para {nombre}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tbody = soup.find('tbody')

        if tbody:
            data = []
            for tr in tbody.find_all('tr'):
                row = [td.text.strip() for td in tr.find_all('td')]
                data.append(row)

            lista_ticker, lista_fecha, lista_hora, lista_precio, lista_vol, lista_ID, lista_flags = [], [], [], [], [], [], []

            for i in data:
                lista_ticker.append(Ticker)
                lista_fecha.append(i[0][:10])
                lista_hora.append(i[0][-8:])
                lista_precio.append(float(i[1].replace(',', '.')))
                # Manejar el caso donde el volumen es '-'
                vol_str = i[2].replace('.', '').replace('- ', '0').replace('-', '0').strip()
                if vol_str == '':
                    vol_str = '0'
                lista_vol.append(int(vol_str))

                if i[5].strip().isdigit():  # Verifica si el valor es un número
                    lista_ID.append(int(i[5]))
                else:
                    lista_ID.append(0)  # Asigna un valor por defecto, como 0, si no es un número

                lista_flags.append('NO' if i[6].strip() == '' else 'SI')

            df = pd.DataFrame({'Ticker': lista_ticker, 'Fecha': lista_fecha, 'Hora': lista_hora, 'Precio': lista_precio,
                               'Volumen': lista_vol, 'CódigoID': lista_ID, 'Flags': lista_flags})

            if not df.empty and not df.isnull().all().all():
                dfbase = pd.concat([dfbase, df], ignore_index=True)

            dfbase.drop_duplicates(inplace=True)

            vol_sum = dfbase['Volumen'].sum()
            barra_de_progreso(int(dfbase['Volumen'].sum()), vol, contador_de_paginas)

            # Verificar si el botón 'Siguiente' está habilitado
            try:
                botones_siguiente = driver.find_elements(By.XPATH, '//a[@href="#"]')
                boton_siguiente = botones_siguiente[-1]
                chivato = boton_siguiente.get_attribute("aria-disabled")
                if chivato == "false":
                  if dfbase['Volumen'].sum() == vol:
                    print(f' -- FALLOS <{fallos}> --', '\033[92m  EL RECUENTO HA SALIDO BIEN\033[0m')
                    print('')
                    break
                  boton_siguiente.click()
                  contador_de_paginas += 1
                  driver.implicitly_wait(15)
                  time.sleep(1)  # Pequeña pausa para cargar la Página                                         #<------------- modificación
                else:
                    if dfbase['Volumen'].sum() != vol:
                      fallos += 1
                      print(f' -- FALLOS <{fallos}> --', '\033[91m  Puta mierda!!! HAY QUE VOLVER A CONTAR\033[0m')
                    else:
                      print(f' -- FALLOS <{fallos}> --', '\033[92m  EL RECUENTO HA SALIDO BIEN\033[0m')
                    print('')
                    break
            except NoSuchElementException:
                if dfbase['Volumen'].sum() != vol:
                      fallos += 1
                # Si no se encuentra el botón 'Siguiente', salir del bucle
                print(f"No se encontró el botón 'Siguiente' para {nombre}")
                break
            except Exception as e:
                if dfbase['Volumen'].sum() != vol:
                  fallos += 1
                print(f"Error al procesar la paginación para {nombre}: {e}")
                break
        else:
            print(f"No se encontraron datos en la Página para {nombre}")
            break

    # Guardar dfbase en CSV
    dfbase.to_csv(ruta_archivo, index=False)

    return dfbase

def imprimir_linea(nombre, vol, vueltas, df0, numero, longitud_total=75):
    texto_dinamico = f"Procesando \033[94m{nombre}\033[0m  con volumen {vol} ----- Vuelta {vueltas} ----- ({len(df0)}/{numero})"
    longitud_dinamica = len(texto_dinamico) - len('\033[94m') - len('\033[0m')  # Restar los caracteres de los códigos ANSI
    longitud_guiones = max(0, longitud_total - longitud_dinamica)  # Guiones que ajustan la longitud
    linea = f"Procesando \033[94m{nombre}\033[0m  con volumen {vol} " + "-" * longitud_guiones + f" Vuelta {vueltas} --- Meta a ({nf}) --- Faltan ({len(df0)}/{numero}) "
    print(linea)

# Procesar cada empresa
while not df0.empty:
  nf = len(df0)-1
  for index, row in df0.iterrows():
      direccion = row['Dirección']
      vol = row['Volumen']
      nombre = row['Empresa']
      imprimir_linea(nombre, vol, vueltas, df0, numero)
      #print(f"\nProcesando \033[94m{nombre}\033[0m  con volumen {vol} ----- Vuelta {vueltas} ----- ({len(df0)}/{numero})")
      df_company = process_company(direccion, vol, nombre, driver)

      if df_company['Volumen'].sum() == vol:
        DATAFRAME = pd.concat([DATAFRAME, df_company], ignore_index=True)
        DATAFRAME.drop_duplicates(inplace=True)
        df0 = df0[df0['Dirección'] != direccion]
        # print(f' -- FALLOS ({fallos}) --', '\033[92m  EL RECUENTO HA SALIDO BIEN\033[0m')
        DATAFRAME.to_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/DATAFRAME.csv', index=False)
      nf = nf - 1
  vueltas += 1

Archivo existente encontrado: /content/drive/MyDrive/Colab Notebooks/Temporal/DATAFRAME.csv. Se utilizará este archivo.
Procesando ACCIONA  con volumen 91566 ------- Vuelta 1 --- Meta a (113) --- Faltan (114/114) 
Timeout al esperar el contenido de la Página para ACCIONA
Procesando ACCIONA ENER  con volumen 320568 - Vuelta 1 --- Meta a (112) --- Faltan (114/114) 
Timeout al esperar el contenido de la Página para ACCIONA ENER
Procesando ACERINOX  con volumen 738546 ----- Vuelta 1 --- Meta a (111) --- Faltan (114/114) 
Timeout al esperar el contenido de la Página para ACERINOX
Procesando ACS  con volumen 275592 ---------- Vuelta 1 --- Meta a (110) --- Faltan (114/114) 
Timeout al esperar el contenido de la Página para ACS
Procesando ADOLFO DGUEZ  con volumen 6 ------ Vuelta 1 --- Meta a (109) --- Faltan (114/114) 


KeyboardInterrupt: 

In [ ]:
# import pandas as pd
# df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/DATAFRAME.csv')
# df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/ENAGAS.csv')
# df3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/CELLNEX.csv')
# df4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/MAPFRE.csv')
# df5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/BA.SABADELL.csv')
# df6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/CELLNEX.csv')
# df7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Temporal/ARCELORMIT..csv')
# DATAFRAME = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# Eliminar las filas donde el valor de la columna 'Ticker' sea 'xxxx'
DATAFRAME = DATAFRAME[DATAFRAME['Ticker'] != 'xxxx']                                                          # <---------- ELIMINANDO LAS FILAS CAGONAS PARA EL APAÑO PATATERO DE LOS ERRORES DE CONCATENACIÓN

# Reiniciar los índices del DataFrame resultante
DATAFRAME = DATAFRAME.reset_index(drop=True)

# Eliminar las filas donde el valor de la columna 'CódigoID' sea igual a 0
DATAFRAME = DATAFRAME[DATAFRAME['CódigoID'] != 0]

DATAFRAME.to_csv('dataIG2.csv', index=False)
DATAFRAME

In [ ]:
DATAFRAME.describe(include='all')

In [ ]:
import datetime

def obtener_mes_actual():
    fecha_actual = datetime.datetime.now()
    meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
             'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    nombre_mes = meses[fecha_actual.month - 1]
    return nombre_mes

mes_actual = obtener_mes_actual()

# Obtener la fecha para el nombre de la carpeta
ff = DATAFRAME['Fecha'].iloc[0]
nombre_carpeta = ff[0:2] + ff[3:5] + ff[8:]

# Crear la carpeta en Google Drive
ruta_carpeta = f"/content/drive/MyDrive/Colab Notebooks/año2025/{mes_actual}/{nombre_carpeta}"
os.makedirs(ruta_carpeta, exist_ok=True)

# Copiar el archivo dataIG2.csv a la carpeta creada
shutil.copy2('dataIG2.csv', os.path.join(ruta_carpeta, 'dataIG2.csv'))

# Eliminar el archivo Temporal
remove("dataIG2.csv")

In [ ]:
# Función para borrar archivos en un directorio
def borrar_archivos(directorio):
    archivos = os.listdir(directorio)
    for archivo in archivos:
        ruta_archivo = os.path.join(directorio, archivo)
        if os.path.isfile(ruta_archivo):
            os.remove(ruta_archivo)

# Borrar archivos en el directorio Temporal
directorio_Temporal = "/content/drive/MyDrive/Colab Notebooks/Temporal"
borrar_archivos(directorio_Temporal)

print("\nArchivos eliminados en", directorio_Temporal)